# Policy Gradient on CartPole

Simple REINFORCE implementation for CartPole balancing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym

np.random.seed(42)

In [ ]:
class PolicyNetwork:
    def __init__(self, state_size=4, action_size=2, lr=0.01):
        self.lr = lr
        self.W1 = np.random.randn(state_size, 16) * 0.1
        self.b1 = np.zeros(16)
        self.W2 = np.random.randn(16, action_size) * 0.1
        self.b2 = np.zeros(action_size)
    
    def forward(self, state):
        h = np.maximum(0, np.dot(state, self.W1) + self.b1)  # ReLU
        logits = np.dot(h, self.W2) + self.b2
        probs = self.softmax(logits)
        return probs, h, logits
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)
    
    def get_action(self, state):
        probs, _, _ = self.forward(state)
        action = np.random.choice(len(probs), p=probs)
        return action, probs
    
    def update(self, states, actions, rewards):
        returns = self.calculate_returns(rewards)
        
        for state, action, G in zip(states, actions, returns):
            probs, h, logits = self.forward(state)
            
            # Policy gradient
            d_logits = probs.copy()
            d_logits[action] -= 1
            d_logits *= G
            
            # Backprop
            dW2 = np.outer(h, d_logits)
            db2 = d_logits
            
            dh = np.dot(d_logits, self.W2.T)
            dh[h <= 0] = 0  # ReLU derivative
            
            dW1 = np.outer(state, dh)
            db1 = dh
            
            # Update weights
            self.W2 -= self.lr * dW2
            self.b2 -= self.lr * db2
            self.W1 -= self.lr * dW1
            self.b1 -= self.lr * db1
    
    def calculate_returns(self, rewards, gamma=0.99):
        returns = []
        G = 0
        for reward in reversed(rewards):
            G = reward + gamma * G
            returns.insert(0, G)
        return returns

In [ ]:
def train_policy_gradient(env, policy, episodes=1000):
    scores = []
    
    for episode in range(episodes):
        state = env.reset()
        states, actions, rewards = [], [], []
        
        while True:
            action, _ = policy.get_action(state)
            next_state, reward, done, _ = env.step(action)
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            
            state = next_state
            
            if done:
                break
        
        policy.update(states, actions, rewards)
        scores.append(sum(rewards))
        
        if episode % 100 == 0:
            avg_score = np.mean(scores[-100:])
            print(f"Episode {episode}: Average Score = {avg_score:.1f}")
    
    return scores

In [ ]:
# Train the agent
env = gym.make('CartPole-v1')
policy = PolicyNetwork(state_size=4, action_size=2, lr=0.01)

scores = train_policy_gradient(env, policy, episodes=800)

# Plot results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(scores, alpha=0.6)
window = 50
moving_avg = [np.mean(scores[max(0, i-window):i+1]) for i in range(len(scores))]
plt.plot(moving_avg, color='red', linewidth=2)
plt.xlabel('Episode')
plt.ylabel('Score')
plt.title('Training Progress')

plt.subplot(1, 2, 2)
recent_scores = scores[-100:]
plt.hist(recent_scores, bins=20, alpha=0.7)
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title('Final 100 Episodes')

plt.tight_layout()
plt.show()

print(f"Final average score: {np.mean(scores[-100:]):.1f}")

In [ ]:
# Test the trained policy
def test_policy(env, policy, episodes=5):
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        steps = 0
        
        while steps < 500:
            action, probs = policy.get_action(state)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        print(f"Test Episode {episode + 1}: Score = {total_reward}, Steps = {steps}")

test_policy(env, policy)
env.close()